In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from ipywidgets import interact


@interact
def sinewave(c = (1.0,3.0), a = 1.0, o = True):

    p = np.linspace(0, 2*math.pi, 100)
    y = a*np.sin(p*c)
    
    plt.plot(p, y)
    if o:
        plt.plot(p, -y)
    plt.ylabel('sin(x)')
    plt.xlabel('x')
    plt.title('Sinwave function')
    return plt.figure()

interactive(children=(FloatSlider(value=2.0, description='c', max=3.0, min=1.0), FloatSlider(value=1.0, descri…